In [2]:
pip install gmaps

     |████████████████████████████████| 1.1 MB 3.9 MB/s eta 0:00:01


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076097 sha256=88394bedc2f4756a4f7b8ca50004ea736eccf9526d8f063d88e8c3237cfc2360
  Stored in directory: /Users/joshzaragoza/Library/Caches/pip/wheels/30/bd/87/d0d731ee712c34a4d6bace8b1dbf8c8932e22a1625b4a55be4
Successfully built gmaps
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [12]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rincon,US,2021-10-30 22:26:14,32.2960,-81.2354,62.56,72,75,9.22
1,1,Cape Town,ZA,2021-10-30 22:23:21,-33.9258,18.4232,63.09,81,0,1.99
2,2,Sitka,US,2021-10-30 22:28:04,57.0531,-135.3300,48.09,61,90,10.36
3,3,Kieta,PG,2021-10-30 22:28:05,-6.2167,155.6333,80.29,81,97,4.21
4,4,Kiunga,PG,2021-10-30 22:28:05,-6.1219,141.2906,76.77,93,61,0.60


In [13]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [19]:
city_data_df.loc[city_data_df["Max Temp"]<0,"Max Temp"]

130    -4.83
132    -0.54
178    -1.91
180    -1.08
254   -10.25
303    -4.40
321    -0.20
330    -2.78
358    -8.45
551    -2.15
Name: Max Temp, dtype: float64

In [21]:
city_data_df.drop(city_data_df[city_data_df["Max Temp"] < 0].index, inplace=True)

In [22]:
city_data_df["Max Temp"].describe()

count    557.000000
mean      59.756122
std       20.308977
min        0.680000
25%       46.040000
50%       62.820000
75%       77.040000
max       92.010000
Name: Max Temp, dtype: float64

In [23]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [24]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [27]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Kieta,PG,2021-10-30 22:28:05,-6.2167,155.6333,80.29,81,97,4.21
4,4,Kiunga,PG,2021-10-30 22:28:05,-6.1219,141.2906,76.77,93,61,0.60
5,5,Poya,NC,2021-10-30 22:27:37,-21.3500,165.1500,79.29,58,3,0.76
16,16,Georgetown,MY,2021-10-30 22:24:41,5.4112,100.3354,78.73,92,40,1.01
19,19,Rikitea,PF,2021-10-30 22:28:11,-23.1203,-134.9692,75.02,85,80,21.12
26,26,Monkey Bay,MW,2021-10-30 22:28:15,-14.0728,34.9201,80.11,50,1,5.19
29,29,Hithadhoo,MV,2021-10-30 22:28:16,-0.6000,73.0833,82.85,75,100,20.67
32,32,Lompoc,US,2021-10-30 22:28:18,34.6391,-120.4579,77.02,78,1,17.27
39,39,Kapaa,US,2021-10-30 22:27:12,22.0752,-159.3190,81.48,85,40,3.00
44,44,Atar,MR,2021-10-30 22:28:23,20.5169,-13.0499,84.40,24,100,5.08


In [28]:
preferred_cities_df.count()

City_ID       169
City          169
Country       168
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kieta,PG,80.29,-6.2167,155.6333,
4,Kiunga,PG,76.77,-6.1219,141.2906,
5,Poya,NC,79.29,-21.3500,165.1500,
16,Georgetown,MY,78.73,5.4112,100.3354,
19,Rikitea,PF,75.02,-23.1203,-134.9692,
26,Monkey Bay,MW,80.11,-14.0728,34.9201,
29,Hithadhoo,MV,82.85,-0.6000,73.0833,
32,Lompoc,US,77.02,34.6391,-120.4579,
39,Kapaa,US,81.48,22.0752,-159.3190,
44,Atar,MR,84.40,20.5169,-13.0499,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEC4XTGKqpKH2nSXEtyEC9I0wDwMHXE0s1mslFm5fat8PRkcqU2M0zpxkXe4APETv4p4Q8az3o5IsHMFTSTFYQjImOwp0jaINxEKV_e0jeNkJHxU1rYiVkXsFZZf88IsegJdPl7LeTJQs80iZaZj1uVV_N7DpuwZBHvuGIHoaNnvtbxk9HhspwgGD0KGzKhrDvAwdSUZGgwOlZGHVR9dtleJLsy1QEXQcITtH0wsW-3zgZHHydNbMlJXRvt9lPGdwlBOrxFcjdGVcJFRLyWf3oE0acnoSU62hZDYFj-XsroLr8t33gEn4Pbqw0R74wrr4i8z5SfSzDLQzO4bKuBSz2icOVhDin8dmXlRYduS4HQIKDa0Hvq8KFssKKhXU1oANr3MQmW7QC-Xj3XvbWF5-vVoAXLOpVvZPgfYP2HPuk0kC9Z5f3iDREF4',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [33]:
len(hotels["results"])

20

In [46]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [49]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Kieta,PG,80.29,-6.2167,155.6333,Uruna Bay Retreat
4,Kiunga,PG,76.77,-6.1219,141.2906,Airport Lodge Kiunga
5,Poya,NC,79.29,-21.3500,165.1500,
16,Georgetown,MY,78.73,5.4112,100.3354,Cititel Penang
19,Rikitea,PF,75.02,-23.1203,-134.9692,People ThankYou
26,Monkey Bay,MW,80.11,-14.0728,34.9201,Monkey Bay Beach Lodge
29,Hithadhoo,MV,82.85,-0.6000,73.0833,Scoop Guest House
32,Lompoc,US,77.02,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
39,Kapaa,US,81.48,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
44,Atar,MR,84.40,20.5169,-13.0499,Oumou Elghoura


In [50]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [54]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))